Setup of neo4j from desktop app:

1. create new local db
    Name: SDM_Lab1_Research
    Username: neo4j
    Password: pass1234

2. Start the db


In [1]:
pip install neo4j


  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.28.1-py3-none-any.whl (312 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pandas as pd
import os
from neo4j import GraphDatabase

In [283]:
NEO4J_URI = "bolt://localhost:7687"  # Default for Neo4j Desktop
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pass1234"  # Change this to your actual password

In [284]:
from neo4j import GraphDatabase

def clear_database():
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("✅ Database cleared!")

clear_database()


✅ Database cleared!


In [285]:
import subprocess

# Run a shell command
result = subprocess.run(['sh', '-c', 'ls ~/Library/Application\ Support/Neo4j\ Desktop/Application/relate-data/dbmss'], capture_output=True, text=True)

# Get the output
print("Output:", result.stdout)


Output: dbms-6f9440a9-c56d-4ebc-b244-90d92b771350



<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:4: SyntaxWarning: invalid escape sequence '\ '
/var/folders/lr/ysc7n3fx6sv02d60ctrl7k_r0000gn/T/ipykernel_44520/3695089776.py:4: SyntaxWarning: invalid escape sequence '\ '
  result = subprocess.run(['sh', '-c', 'ls ~/Library/Application\ Support/Neo4j\ Desktop/Application/relate-data/dbmss'], capture_output=True, text=True)


In [286]:
# copy the output above in the path here

IMPORT_FOLDER = os.path.expanduser("~/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/")




- Nodes:

1. Paper: doi, title, abstract, year, # citations
2. Author: author id, name
3. Conference: conf id, name 
4. Edition: edition id, edition year, conference id, proceeding name , city
5. Journal: id, name
6. Keyword/Topic: id, name

- Relationship:
1. author - writes -> paper : author id, paper doi
2. author - corresponding author of -> paper : : author id, paper doi
3. paper - cites -> paper : DOI1, DOI2
4. paper - about -> keyword: doi, keyword id
5. paper - published in ({volume}) -> journel : doi, journal id , volume number , pages, year
6. paper - published in -> conference/workshop edition : doi, edition id , pages
7. edition - of -> conference/workshop : edition id, conf id

In [287]:
# Paper nodes csv 

import pandas as pd

papers_df = pd.read_csv("papers.csv")
referenced_df = pd.read_csv("referenced_papers.csv")

concatenated_df = pd.concat([papers_df, referenced_df], ignore_index=True)

concatenated_df = concatenated_df.drop_duplicates(subset=["DOI"], keep="first")

selected_columns = ["DOI", "Title", "Abstract", "Year", "publicationDate", "Citations", "References", "URL"]
filtered_df = concatenated_df[selected_columns]

filtered_df.loc[:, 'Abstract'] = filtered_df['Abstract'].str.replace(r'\"\"', '', regex=True)

# Save the cleaned DataFrame to a new CSV file
filtered_df.to_csv("Paper_nodes.csv", index=False)

print("Concatenated data without duplicates has been saved to 'Paper_nodes.csv'")


Concatenated data without duplicates has been saved to 'Paper_nodes.csv'


In [288]:
# Author nodes csv 

authors_df = pd.read_csv("authors.csv")
ref_authors_df = pd.read_csv("ref_authors.csv")


concatenated_df = pd.concat([authors_df, ref_authors_df], ignore_index=True)

concatenated_df = concatenated_df.drop_duplicates(subset=["Author_ID"], keep="first")


# Save the cleaned DataFrame to a new CSV file
concatenated_df.to_csv("Author_nodes.csv", index=False)

print("Concatenated data without duplicates has been saved to 'Author_nodes.csv'")


Concatenated data without duplicates has been saved to 'Author_nodes.csv'


In [260]:
# # Separate Journal and Conference 

# journal_papers_df = papers_df[papers_df["publicationType"]=="journal"]
# journal_ref_papers_df = referenced_df[referenced_df["publicationType"]=="journal"]

# journal_papers_concat = pd.concat([journal_papers_df,journal_ref_papers_df], ignore_index=True)

# conf_papers_df = papers_df[papers_df["publicationType"]=="conference"]
# conf_ref_papers_df = referenced_df[referenced_df["publicationType"]=="conference"]

# conf_papers_concat = pd.concat([conf_papers_df,conf_ref_papers_df], ignore_index=True)

In [255]:
# ########### Handle 
# referenced_df[referenced_df["publicationType"]=="Unknown"]

,Paper_ID,DOI,Title,Abstract,Venue,publicationType,publication_type_2,journal_name,pages,volume,Year,publicationDate,Citations,References,Authors,Author_Ids,URL,s2FieldsOfStudys,Corresponding_Author_ID
2,286519b404064fe4e52f1dc5483596fe683aed03,10.1109/GlobConHT56829.2023.10087780,Machine Learning-based Electric Vehicle User B...,NaN,2023 IEEE IAS Global Conference on Renewable E...,Unknown,['Conference'],2023 IEEE IAS Global Conference on Renewable E...,1-6,NaN,2023,2023-03-11 00:00:00,4,20,"Aakash Lilhore, Kavita Kiran Prasad, V. Agarwal","2213915423, 31624194, 2113266595",https://www.semanticscholar.org/paper/286519b4...,"Computer Science, Engineering, Environmental S...",2213915423
12,8c5e5df92c119bff25095b79ea37b1f62ab0a511,10.1109/DASC-PICom-CBDCom-CyberSciTech49142.20...,Detection of DoH Tunnels using Time-series Cla...,NaN,"2020 IEEE Intl Conf on Dependable, Autonomic a...",Unknown,['JournalArticle'],"2020 IEEE Intl Conf on Dependable, Autonomic a...",63-70,NaN,2020,2020-08-01 00:00:00,130,32,"Mohammadreza MontazeriShatoori, L. Davidson, G...","2023739110, 2014749334, 2269790, 2037871",https://www.semanticscholar.org/paper/8c5e5df9...,Computer Science,2023739110
13,e51f66d1ee807dd0edd5fa47fc2e3a707707435d,10.1109/TSMC.2019.2905253,Distributed Resilient Filtering for Power Syst...,NaN,"IEEE Transactions on Systems, Man, and Cyberne...",Unknown,['JournalArticle'],"IEEE Transactions on Systems, Man, and Cyberne...",1688-1697,49,2019,2019-04-02 00:00:00,229,45,"Wei Chen, Derui Ding, Hongli Dong, Guoliang Wei","2154941306, 1795716, 2597875, 2255389230",https://www.semanticscholar.org/paper/e51f66d1...,"Computer Science, Engineering",2154941306
15,4880ef516028dd598fe9339429b36658530fd3c2,10.1002/anie.202415056,Inverse Design of Singlet‐Fission Materials wi...,Abstract Singlet fission has shown potential f...,Angewandte Chemie,Unknown,['JournalArticle'],Angewandte Chemie (International Ed. in English),NaN,64,2024,2024-09-25 00:00:00,1,9,"Luca Schaufelberger, J. T. Blaskovits, Rubén L...","2322850680, 2255859554, 32427390, 2321647360, ...",https://www.semanticscholar.org/paper/4880ef51...,"Materials Science, Medicine",2322850680
17,a129b8ce13b257f19384c4b7ca6c207c9343e366,10.1038/s42256-024-00815-9,Equivariant 3D-conditional diffusion model for...,NaN,Nat. Mac. Intell.,Unknown,['JournalArticle'],Nat. Mac. Intell.,417-427,6,2024,2024-04-11 00:00:00,53,28,"Ilia Igashov, Hannes Stärk, Clément Vignac, Ar...","1677188150, 2312039321, 1404062244, 89961757, ...",https://www.semanticscholar.org/paper/a129b8ce...,"Chemistry, Computer Science",1677188150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,38b83ab5798bf457454a14230b6f22892d91b92d,10.1109/3ICT.2018.8855790,Data Mining for Intelligent Academic Advising ...,NaN,2018 International Conference on Innovation an...,Unknown,['Conference'],2018 International Conference on Innovation an...,1-5,NaN,2018,2018-11-01 00:00:00,2,25,"A. Ghanem, H. Alobaidy","35391618, 8134960",https://www.semanticscholar.org/paper/38b83ab5...,"Computer Science, Education",35391618
1894,91e71c006cd6712aee74fe972648c80443a366e7,10.2307/1269814,Missing Data,NaN,Definitions,Unknown,NaN,Definitions,NaN,NaN,2020,2020-02-02 00:00:00,1144,0,"Subir Ghosh, R. Little, D. Rubin","2324699484, 2324436256, 2324436398",https://www.semanticscholar.org/paper/91e71c00...,Computer Science,2324699484
1895,ffc1c6d02f916475cb7d0793f7c825b6d8f7713e,10.1016/J.COMPEDU.2019.04.009,Integrating machine learning into item respons...,NaN,Comput. Educ.,Unknown,['JournalArticle'],Comput. Educ.,91-103,137,2019,2019-08-01 00:00:00,87,71,"Konstantinos Pliakos, Seang-Hwane Joo, Jung Ye...","1744769, 38511894, 48490753, 35119306, 2379840...",https://www.semanticscholar.org/paper/ffc1c6d0...,Computer Science,1744769
1901,6603e3fd463a287416a086116cb6e4068e9d380b,10.1109/ISRCS.2013.6623770,Investigating the application of moving target...,NaN,2013 6th International Symposium on Resilient ...,Unknown,NaN,2013 6th

In [289]:
import shutil
shutil.copy("Paper_nodes.csv", IMPORT_FOLDER)
shutil.copy("Author_nodes.csv", IMPORT_FOLDER)
shutil.copy("fields_of_study.csv", IMPORT_FOLDER)
shutil.copy("conferences.csv", IMPORT_FOLDER)
shutil.copy("journals.csv", IMPORT_FOLDER)
shutil.copy("conference_editions.csv", IMPORT_FOLDER)



'/Users/marwasulaiman/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/conference_editions.csv'

In [290]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

session = driver.session() 

# Load papers 
query =     """
            LOAD CSV WITH HEADERS FROM "file:///Paper_nodes.csv" AS row
            CREATE (:Paper { DOI : row.DOI, title : row.Title, abstract : row.Abstract, year : toInteger(row.Year), 
            publicationDate : date(substring(row.publicationDate, 0, 10)), citationCount : toInteger(row.Citations), referenceCount : toInteger(row.References),
            URL : row.URL})
            """
session.run(query)



In [291]:
# Load Authors 

query =     """
            LOAD CSV WITH HEADERS FROM "file:///Author_nodes.csv" AS row
            CREATE (:Author { ID : toInteger(row.Author_ID), Name: row.Author_Name})
            """
session.run(query)

In [292]:
# Load Keywords 

query =     """
            LOAD CSV WITH HEADERS FROM "file:///fields_of_study.csv" AS row
            CREATE (:Keyword { ID : row.Field_ID, Name: row.Field_Name})
            """
session.run(query)

In [293]:
# Load conferences
query =     """
            LOAD CSV WITH HEADERS FROM "file:///conferences.csv" AS row
            CREATE (:Conference { ID : row.ID, Name: row.Name, URL: row.url})
            """
session.run(query)

In [294]:
# Load Editions
query =     """
            LOAD CSV WITH HEADERS FROM "file:///conference_editions.csv" AS row
            CREATE (:Conference_Edition { ID : row.Edition_ID, Venue_ID: row.Venue_ID, Year:row.Year, Proceeding: row.Proceeding_Name})
            """
session.run(query)

In [295]:
# Load Journals
query =     """
            LOAD CSV WITH HEADERS FROM "file:///journals.csv" AS row
            CREATE (:Journal { ID : row.ID, Name: row.Name, ISSN: row.issn, URL: row.url})
            """
session.run(query)

Relationships

In [296]:
shutil.copy("author_paper_relationship.csv", IMPORT_FOLDER)
shutil.copy("ref_author_paper_relationship.csv", IMPORT_FOLDER)
shutil.copy("paper_field_relationship.csv", IMPORT_FOLDER)
shutil.copy("ref_paper_field_relationship.csv", IMPORT_FOLDER)


'/Users/marwasulaiman/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/ref_paper_field_relationship.csv'

In [297]:
query =    """LOAD CSV WITH HEADERS FROM 'file:///author_paper_relationship.csv' AS row
            MATCH (a:Author {ID: toInteger(row.Author_ID)}), (p:Paper {DOI: row.DOI})
            MERGE (a)-[w:writes]->(p)
            """
session.run(query)

In [298]:
query =    """LOAD CSV WITH HEADERS FROM 'file:///author_paper_relationship.csv' AS row
            MATCH (a:Author {ID: toInteger(row.Author_ID)}), (p:Paper {DOI: row.DOI})
            WHERE toBoolean(row.Corresponding) = True
            MERGE (a)-[:CORRESPONDING_AUTHOR_OF]->(p);  // Creating a separate edge for corresponding authors
            """
session.run(query)